In [1]:
import yaml
from GeneralMethod import transforRepPattern, findElementValue, uniRep


# set variable to read the mined patterns
sam_types = [] # resource types
sam_allentries = [] # configuration entries
sam_correlationleftCorse = [] # coarse-gained dependencies: left elements
sam_correlationrightCorse = [] # coarse-gained dependencies: right elements
sam_correlationleft = [] # fine-gained dependencies: left elements
sam_correlationright = [] # fine-gained dependencies: right elements
sam_keyMapValue_key = [] # mappings about the confiration entry and values, confiration entry
sam_keyMapValue_value = [] # mappings about the confiration entry and values, confiration entry values

In [2]:
# read the learned patterns
def readKnowledge():
    for line in open("Patterns/{}.txt".format("sam_types")):
        line = line.replace("\n","")
        sam_types.append(line)
    
    for line in open("Patterns/{}.txt".format("sam_types_add")):
        line = line.replace("\n","")
        sam_types.append(line)
    
    for line in open("Patterns/{}.txt".format("sam_allentries")):
        line = line.replace("\n","")
        sam_allentries.append(line)
    for line in open("Patterns/{}.txt".format("sam_allentries_add")):
        line = line.replace("\n","")
        sam_allentries.append(line)

    for line in open("Patterns/{}.txt".format("sam_correlationleftFine")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationleft.append(newlist)
    for line in open("Patterns/{}.txt".format("sam_correlationrightFine")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationright.append(newlist)


    for line in open("Patterns/{}.txt".format("sam_correlationleftCoarse")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationleftCorse.append(newlist)
    for line in open("Patterns/{}.txt".format("sam_correlationrightCoarse")):
        line = line.replace("\n","")
        line_tmp = line.split("+++")
        newlist = []
        for i in line_tmp:
            newlist.append(i)
        sam_correlationrightCorse.append(newlist)
    keyMapValue = findElementValue()
    # print(sam_keyMapValue)
    for key in keyMapValue.keys():
        
        sam_keyMapValue_key.append(key)
        sam_keyMapValue_value.append(keyMapValue[key])


readKnowledge()

update cnfiguraiton entry values


In [3]:
#--------------------------misconfugraiton detection----------

# 1. resoure type check
def samTypeCheck(flat_config):
    # print("resoure type check")
   
    problem_types = []
    for key_i in flat_config.keys():
        key_i_tmp = key_i.split(".")
        if key_i_tmp[-1] == "Type":
            if flat_config[key_i] not in sam_types:
                problem_types.append(flat_config[key_i])
    problem_types = list(set(problem_types))
    print("1--》Resource type violation：{}".format(problem_types))
    

In [4]:
# 2. configuration entry check
def samEntryCheck(flat_config):
   
    problem_entries = []
    specialProcessEntry = ["Resources.PHAWS::Serverless::Function.Properties.Environment.Variables", "Resources.PHAWS::Serverless::Api.Properties.Variables", "Properties.Parameters", "Resources.PHAWS::Serverless::Api.Properties.GatewayResponses", "Resources.PHAWS::Serverless::Api.Properties.DefinitionBody", "Resources.PHAWS::Serverless::Api.Properties.Auth.Authorizers", "Resources.PHAWS::Serverless::Function.Properties.Tags", "Globals.Function.Environment.Variables"]
    specialSuffix = ["Sub", "If", "ImportValue", "GetAtt", "Join"]

    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)   

    for key_i in flat_config_key_new:
        if key_i not in sam_allentries:

            key_i_tmp = key_i.split(".")
            checkflag0 = checkListEntryInValue(key_i_tmp[-1], specialSuffix)
            
            checkflag00 = 1000 

            if len(key_i_tmp)>1:
                checkflag00 = checkListEntryInValue(key_i_tmp[-2], specialSuffix)
        
            if checkflag0 != 0:
                
                key_i_new = ".".join(key_i_tmp[0:-1])
                if checkflag00 > 0 and checkflag00 < 1000: 
                   
                    key_i_new = ".".join(key_i_tmp[0:-2])
            else:
                
                key_i_new = ".".join(key_i_tmp)
            
            checkflag1 = CustomcheckEntryInList(key_i_new, sam_allentries)
            checkflag2 = checkListEntryInValue(key_i_new, specialProcessEntry)
        
            if checkflag1 == 0 and checkflag2 == 0:
                if "Fn::Transform" in key_i_tmp:
                    if ("Fn::Transform.Name" not in ".".join(key_i_tmp)) and ("Fn::Transform.Parameters" not in ".".join(key_i_tmp)):
                        problem_entries.append(".".join(key_i_tmp)+"-----lacking Name properties")
                else:
                    problem_entries.append(".".join(key_i_tmp))

            problem_entries = list(set(problem_entries))

    problem_entries = list(set(problem_entries))


    from collections import defaultdict
    grouped_numbers = defaultdict(list)

    for key_tmp in flat_config_key_new:
        key_tmp_tmp = key_tmp.split(".")
        if key_tmp_tmp[0] == "Resources" and key_tmp_tmp[-1]!="Type":
            type_tmp = key_tmp_tmp[1]
            grouped_numbers[type_tmp].append(key_tmp)

    for group, numbers in grouped_numbers.items():
       
        count_in_other_list = sum(1 for number in numbers if number in problem_entries)
        if count_in_other_list >= (len(numbers) * 0.7):
            print(f"1.2--》Resource type violation：{group}type problems")
        
    print("2--》Configuration entry violation: sam_entry：{}".format(problem_entries))

    return problem_entries

def checkListEntryInValue(value, checklist):
    checkflag = 0
    for k in checklist:
        if k not in value:
            checkflag = checkflag + 0
        else:
            checkflag = checkflag + 1
    return checkflag

def CustomcheckEntryInList(value, checklist):
    # print(value)
    checkflag = 0
    value_new = value.split(".")
    for k in checklist:
        k_new = k.split(".")
        if len(k_new) == len(value_new):
            newflag = 0 
            for index in range(len(value_new)):
                if value_new[index] == k_new[index]:
                    newflag = newflag + 0
                else:
                    newflag = newflag + 1


            if newflag > 0:
                checkflag = checkflag + 0
            else:
                checkflag = checkflag + 1
    return checkflag


In [5]:
# 3. configuration entry value check
def  samKeyvalueCheck(flat_config, problem_entries):


    moreRef = []
    lessRef = []
    for i in problem_entries:
        if ".Ref" in i:
            tmp = i.replace(".Ref", "")
            if tmp in sam_allentries:
                moreRef.append(i)

        if ".Ref" not in i:
            tmp = i+".Ref"
            if tmp in sam_allentries:
                lessRef.append(i)
    print("3.1--》***Suspicious configuration entry values, incorrect use of the Ref form***** {}".format(moreRef))
    print("3.2--》***Suspicious configuration entry values, Ref form not used***** {}".format(lessRef))


    
    problem_sam_key = []
    true_sam_key = []
    problem_sam_value = []
    
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)


    for index in range(len(flat_config_key_new)):

        for key_index in range(len(sam_keyMapValue_key)):
            
            if flat_config_key_new[index] == sam_keyMapValue_key[key_index]:

                if (flat_config_value_new[index] not in sam_keyMapValue_value[key_index]) and is_contained(flat_config_value_new[index], sam_keyMapValue_value[key_index]) == False:
                    
                    problem_sam_key.append(flat_config_key_new[index])
                    true_sam_key.append(sam_keyMapValue_value[key_index])
                    problem_sam_value.append(flat_config_value_new[index])

    for i in range(len(problem_sam_key)):
        print("3.3--》 Suspicious configuration entry values, specifically, please check")
        print("3.3.1--》***Suspicious configuration entry***** {}".format(problem_sam_key[i]))
        print("3.3.2--》***Corresponding values***** {}****Type is {}****Whether there is a type equality: {}".format(problem_sam_value[i], type(problem_sam_value[i]), any(type(problem_sam_value[i]) == type(element) for element in true_sam_key[i])))
        if problem_sam_key[i] == "Resources.PHAWS::Serverless::Function.Properties.MemorySize":
            if problem_sam_value[i] < 128 or problem_sam_value[i] > 10240:
                print("The function memory size is the value between 128 MB and 10240 MB.")
        if problem_sam_value[i] == None:
            print("The configuration template has null values in YAML.")

        print("3.3.3--》***exsiting values contain******")
        k = 0
        for j in true_sam_key[i]:
            k=k+1
            print("===============The value is {}, the type is {}. Whether or not it is of the same type as the give: {}, it is ******\n{}".format(k,type(j),type(j)==type(problem_sam_value[i]), j))
            if k == 10:
                break

def is_hashable(obj):
    try:
        hash(obj)
    except TypeError:
        return False
    return True

def is_contained(A, B):
    if not isinstance(A, list) or not all(is_hashable(x) for x in A):
        return False

    set_A = set(A)
    for item in B:
        if isinstance(item, list) and all(is_hashable(x) for x in item):
            if set(item) == set_A:
                return True
    return False

In [6]:

# 4. configuration dependency check
def samRuleCheck(flat_config):
    # coarse-gained configuration dependency check
    samRuleCheckCorse(flat_config)
    # fine-gained configuration dependency check
    samRuleCheckFine(flat_config)


def samRuleCheckCorse(flat_config):
    problem_sam_ruleright = []
    entryfile = ReptoEntryvalueRepSAMCorse(flat_config)
    entryfileset = set(entryfile)
   
    for index, leftvalue in enumerate(sam_correlationleftCorse):

        leftvalue_tmp = set(leftvalue)
        if leftvalue_tmp.issubset(entryfileset):

            for rightvalue in sam_correlationrightCorse[index]:
                if rightvalue not in entryfile:
        
                    print("----")
                    print("+++".join(leftvalue))
                    problem_sam_ruleright.append(rightvalue)
    problem_sam_ruleright = list(set(problem_sam_ruleright))
    print("4.1--》Configuration dependency violation-coarse-gained dependencies-sam_rule: no {}".format(problem_sam_ruleright))


def samRuleCheckFine(flat_config):



    problem_sam_ruleright = []
    entryfile = ReptoEntryvalueRepSAM(flat_config)
    # print(entryfile)
    entryfileset = set(entryfile)

    for index, leftvalue in enumerate(sam_correlationleft):

        leftvalue_tmp = set(leftvalue)
        if leftvalue_tmp.issubset(entryfileset):
 
            for rightvalue in sam_correlationright[index]:
                if rightvalue not in entryfile:
    
                    print("----")
                    print(leftvalue)
                    problem_sam_ruleright.append(rightvalue)
    problem_sam_ruleright = list(set(problem_sam_ruleright))
    print("4.2--》Configuration dependency violation-fine-gained dependencies-sam_rule: no {}".format(problem_sam_ruleright))



def ReptoEntryvalueRepSAMCorse(flat_config):

    entryfile = []
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)
    for i in range(len(flat_config_key_new)):
        tmp = "{}".format(flat_config_key_new[i])  
        entryfile.append(tmp)
    entryfile = list(set(entryfile)) 
    return entryfile   


def ReptoEntryvalueRepSAM(flat_config):
    entryfile = []
    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)
    for i in range(len(flat_config_key_new)):
        tmp = "{}={}".format(flat_config_key_new[i], flat_config_value_new[i])  
        entryfile.append(tmp)
    entryfile = list(set(entryfile)) 
    return entryfile    



In [7]:
# 5. special constraint check
def samValueRuleCheck(flat_config):

    problem_sam_valuerule = []

    flat_config_key_new, flat_config_value_new = transforRepPattern(flat_config)
   
    # 1. Check that the value of VisibilityTimeout is not less than the Timeout value defined by Serverless Functions
    sqstimeout = "no"
    functiontimeout = 0
    flag = 0
    packagetypeValue = ""
    
    for index in range(len(flat_config_key_new)):
        if flat_config_key_new[index] == "Resources.PHAWS::SQS::Queue.Properties.VisibilityTimeout":
            sqstimeout = int(flat_config_value_new[index])
        if flat_config_key_new[index] == "Resources.PHAWS::Serverless::Function.Properties.Timeout" or flat_config_key_new[index] == "Globals.Function.Timeout":
            
            functiontimeout = int(flat_config_value_new[index])
        if flat_config_key_new[index] == "Resources.PHAWS::Serverless::Function.Type":
            flag = 1
        if flat_config_key_new[index] == "Resources.PHAWS::Serverless::Function.Properties.PackageType":
            packagetypeValue = flat_config_value_new[index]
        

    if sqstimeout != "no" and functiontimeout != 0 and sqstimeout <= functiontimeout:
        problem_sam_valuerule.append(["Resources.PHAWS::SQS::Queue.Properties.VisibilityTimeout", "Resources.PHAWS::Serverless::Function.Properties.Timeout", "Fail to satisfy a constraint relationship about timeout"])
    
    

    # 2.  Checking that Function's Timeout is not set too small, which by default is 3 seconds when not set, also suggests that there may be a problem. 
    # Because of the small timeout setting, there is no indication that there is a problem when checking the value of the configuration item.
    if flag == 1 and functiontimeout <= 3:
        problem_sam_valuerule.append(["The timeout value of serverless functions may be too small"])


    # 3. Check the Api definition for event under serverless function, possible issues
    if ("Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Type" in flat_config_key_new) and ("Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Properties.RestApiId.Ref" not in flat_config_key_new) :
        if "Resources.PHAWS::Serverless::Api.Type" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Properties.RestApiId.Ref may not be defined" ])
        else:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Properties.RestApiId.Ref may not be defined and Resources.PHAWS::Serverless::Api.Type may not be defined" ])
    
    if ("Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Type" in flat_config_key_new) and ("Resources.PHAWS::Serverless::Function.Properties.Events.PHApi.Properties.RestApiId.Ref" in flat_config_key_new) :
        if "Resources.PHAWS::Serverless::Api.Type" not in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Api.Type may not be defined" ])
        
    
    # 4. Check the HttpApi definition for event under serverless function, possible issues
    if ("Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Type" in flat_config_key_new) and ("Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Properties.ApiId.Ref" not in flat_config_key_new) :
        if "Resources.PHAWS::Serverless::HttpApi.Type" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Properties.ApiId may not be defined" ])
        else:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Properties.ApiId may not be defined and Resources.PHAWS::Serverless::HttpApi.Type may not be defined" ])
    if ("Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Type" in flat_config_key_new) and ("Resources.PHAWS::Serverless::Function.Properties.Events.PHHttpApi.Properties.ApiId.Ref" in flat_config_key_new) :
        if "Resources.PHAWS::Serverless::HttpApi.Type" not in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::HttpApi.Type is not defined" ])

    # 5. Checking the PackageType constraints for serverless functions
    # If PackageType is set to Image type, properties for Runtime, Handler and Layers are not present, will define specific ones in dockerfile. 
    # However, the ImageUri attribute needs to be present, there is no CodeUri or InlineCode attribute
    if packagetypeValue == "Image":

        if "Resources.PHAWS::Serverless::Function.Properties.Runtime" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Runtime is not required" ])
        if "Resources.PHAWS::Serverless::Function.Properties.Handler" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Handler is not required" ])
        if "Resources.PHAWS::Serverless::Function.Properties.Layers" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.Layers is not required" ])
        if "Resources.PHAWS::Serverless::Function.Properties.CodeUri" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.CodeUri is not required" ])
        if "Resources.PHAWS::Serverless::Function.Properties.InlineCode" in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.InlineCode is not required" ])
    if "Resources.PHAWS::Serverless::Function.Properties.ImageUri.Ref" in flat_config_key_new:
        
        if "Resources.PHAWS::Serverless::Function.Properties.PackageType" not in flat_config_key_new:
            problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.PackageType is not defined" ])
        else:
            if packagetypeValue != "Image":
                problem_sam_valuerule.append(["Resources.PHAWS::Serverless::Function.Properties.PackageType should be Image type"])


    print("5--》Special constraint violation-sam_valuerule：{}".format(problem_sam_valuerule))



In [8]:
# for the tested configuration file
filePath = "../Evaluation/RealWorldMisconfigurations/TEST21.yaml"

try:
    # transform uniform representation
    flat_config = uniRep(filePath)
    # 1. resource type check
    samTypeCheck(flat_config)
    # 2. configuration entry check
    problem_entries = samEntryCheck(flat_config)
    # 3. configuration entry value check
    samKeyvalueCheck(flat_config, problem_entries)
    # 4. configuration dependency check
    samRuleCheck(flat_config)
    # 5. special constraint check
    samValueRuleCheck(flat_config)

    # pass
except Exception as e:
    print("Configuration file parse problem, please check grammer for: {}".format(e))
    # pass

1--》Resource type violation：[]
2--》Configuration entry violation: sam_entry：['Resources.PHAWS::Serverless::Function.Properties.Events.PHS3.Condition']
3.1--》***Suspicious configuration entry values, incorrect use of the Ref form***** []
3.2--》***Suspicious configuration entry values, Ref form not used***** []
3.3--》 Suspicious configuration entry values, specifically, please check
3.3.1--》***Suspicious configuration entry***** Parameters.PHString.Default
3.3.2--》***Corresponding values***** no****Type is <class 'str'>****Whether there is a type equality: True
3.3.3--》***exsiting values contain******
===============The value is 1, the type is <class 'str'>. Whether or not it is of the same type as the give: True, it is ******
'*'
===============The value is 2, the type is <class 'str'>. Whether or not it is of the same type as the give: True, it is ******
black
===============The value is 3, the type is <class 'str'>. Whether or not it is of the same type as the give: True, it is ******